# Solar Cell

## Photovoltaic Device Physics

Solar cells convert light energy into electrical energy using the photovoltaic effect. Understanding the device physics is essential for optimizing efficiency.

**Learning Objectives:**
- Understand solar cell operation
- Simulate dark I-V characteristics with PADRE
- Analyze carrier generation and recombination effects
- Study the impact of carrier lifetime and surface recombination

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## 1. Solar Cell Physics

### 1.1 Structure

```
    Light
      |
      v
  =========== Front Contact
  | N+ Emitter (thin)
  |-----------| Junction
  |           |
  | P Base    | (thick absorber)
  |           |
  =========== Back Contact
```

### 1.2 Key Parameters

- **Open-circuit voltage** ($V_{oc}$):
  $$V_{oc} = \frac{kT}{q}\ln\left(\frac{I_L}{I_0} + 1\right)$$

- **Short-circuit current** ($I_{sc}$): Equal to photogenerated current $I_L$

- **Fill Factor** (FF):
  $$FF = \frac{P_{max}}{V_{oc} \cdot I_{sc}}$$

- **Efficiency**:
  $$\eta = \frac{P_{max}}{P_{in}} = \frac{V_{oc} \cdot I_{sc} \cdot FF}{P_{in}}$$

- **Diffusion Length**:
  $$L = \sqrt{D \tau}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_solar_cell

# Physical constants
q = 1.6e-19
kT = 0.0259  # eV at 300K

# Material parameters for silicon
D_n = 35      # Electron diffusion coefficient (cm^2/s)
D_p = 12      # Hole diffusion coefficient (cm^2/s)

print("Solar Cell Parameters:")
print("="*40)
print(f"Thermal voltage VT = {kT*1000:.1f} mV")
print(f"Silicon bandgap = 1.12 eV")
print(f"Typical Voc ~ 0.5-0.7 V for Si")

---

## 2. Creating and Running a Solar Cell Simulation

Let's create an N-on-P solar cell structure and run PADRE to examine its characteristics.

In [ ]:
# Create an N-on-P solar cell
sim_solar = create_solar_cell(
    # Geometry
    emitter_depth=0.5,      # 500nm emitter
    base_thickness=200.0,   # 200um base (absorber)
    device_width=1.0,
    
    # Mesh
    nx=3,
    ny=100,
    
    # Doping
    emitter_doping=1e19,    # N+ emitter
    base_doping=1e16,       # P base
    device_type='n_on_p',
    
    # Models
    temperature=300,
    srh=True,               # SRH recombination
    auger=True,             # Auger recombination
    
    # Material parameters
    taun0=1e-5,             # Electron lifetime (10 us)
    taup0=1e-5,             # Hole lifetime (10 us)
    
    # Surface recombination
    front_surface_velocity=1e4,   # Front SRV (cm/s)
    back_surface_velocity=1e7,    # Back SRV (cm/s)
    
    # Output
    log_bands_eq=True
)

print("Solar Cell Configuration:")
print("="*40)
print("Structure: N-on-P")
print("Emitter: N+ 1e19 cm⁻³, 0.5 μm")
print("Base: P 1e16 cm⁻³, 200 μm")
print("Carrier lifetimes: 10 μs")

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_solar.generate_deck())

In [ ]:
# Run equilibrium simulation
print("Running solar cell equilibrium simulation...")
result_eq = sim_solar.run()

if result_eq.returncode == 0:
    print("Simulation completed successfully!")
    print(f"Output directory: {sim_solar.working_dir}")
else:
    print("Simulation failed!")
    print(result_eq.stderr)

In [ ]:
# Plot equilibrium band diagram
print("\nAvailable outputs:")
print(sim_solar.outputs.summary())

In [ ]:
# Plot band diagram
try:
    sim_solar.plot_band_diagram(title="Solar Cell Equilibrium Band Diagram")
except Exception as e:
    print(f"Could not plot band diagram: {e}")

---

## 3. Dark I-V Characteristics

The dark I-V characteristic shows the diode behavior of the solar cell without illumination. This reveals the recombination mechanisms.

In [ ]:
# Dark I-V simulation
sim_dark = create_solar_cell(
    emitter_depth=0.5,
    base_thickness=200.0,
    device_width=1.0,
    emitter_doping=1e19,
    base_doping=1e16,
    device_type='n_on_p',
    
    temperature=300,
    srh=True,
    auger=True,
    
    taun0=1e-5,
    taup0=1e-5,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="dark_iv",
    
    # Forward bias sweep
    forward_sweep=(0.0, 0.7, 0.02)
)

print("Dark I-V Simulation Configuration:")
print("="*40)
print("Voltage sweep: 0 to 0.7V")
print("Step size: 0.02V")

In [ ]:
# Run the dark I-V simulation
print("Running dark I-V simulation...")
result_dark = sim_dark.run()

if result_dark.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")
    print(result_dark.stderr)

In [ ]:
# Plot the dark I-V characteristic
try:
    iv_data = sim_dark.get_iv_data()
    V_dark, I_dark = iv_data.get_iv_data(electrode=2)
    
    V_dark = np.array(V_dark)
    I_dark = np.abs(np.array(I_dark))
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Linear scale
    ax1.plot(V_dark, I_dark * 1e3, 'b-o', linewidth=2, markersize=4, label='PADRE Simulation')
    ax1.set_xlabel('Voltage (V)', fontsize=12)
    ax1.set_ylabel('Current (mA)', fontsize=12)
    ax1.set_title('Dark I-V (Linear Scale)', fontsize=14)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Semi-log scale
    ax2.semilogy(V_dark, I_dark, 'b-o', linewidth=2, markersize=4, label='PADRE Simulation')
    ax2.set_xlabel('Voltage (V)', fontsize=12)
    ax2.set_ylabel('Current (A)', fontsize=12)
    ax2.set_title('Dark I-V (Semi-log Scale)', fontsize=14)
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Extract diode parameters
    print("\nDark I-V Analysis:")
    print("="*40)
    
    # Fit to extract ideality factor
    mask = (V_dark > 0.2) & (V_dark < 0.5) & (I_dark > 0)
    if np.sum(mask) > 2:
        coeffs = np.polyfit(V_dark[mask], np.log(I_dark[mask]), 1)
        n_ideality = 1 / (kT * coeffs[0])
        I0 = np.exp(coeffs[1])
        print(f"Ideality factor: n = {n_ideality:.2f}")
        print(f"Saturation current: I0 = {I0:.3e} A")
    
except Exception as e:
    print(f"Could not plot dark I-V: {e}")

---

## 4. Effect of Carrier Lifetime

Carrier lifetime determines the diffusion length and strongly affects solar cell efficiency. Let's explore this with PADRE simulations.

In [ ]:
# Compare different lifetimes
lifetimes = [1e-7, 1e-6, 1e-5]  # seconds
lifetime_results = {}

print("Lifetime Comparison:")
print("="*50)

for tau in lifetimes:
    # Calculate diffusion length
    L_n = np.sqrt(D_n * tau) * 1e4  # Convert to um
    print(f"\nτ = {tau:.0e} s → L = {L_n:.1f} μm...")
    
    sim = create_solar_cell(
        emitter_depth=0.5,
        base_thickness=200.0,
        emitter_doping=1e19,
        base_doping=1e16,
        device_type='n_on_p',
        temperature=300,
        srh=True,
        taun0=tau,
        taup0=tau,
        log_iv=True,
        iv_file=f"dark_tau{int(-np.log10(tau))}",
        forward_sweep=(0.0, 0.7, 0.02)
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            V, I = iv.get_iv_data(electrode=2)
            lifetime_results[tau] = (np.array(V), np.abs(np.array(I)), L_n)
            print(f"  Simulation successful")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Simulation failed")

In [ ]:
# Plot lifetime comparison
if lifetime_results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    colors = ['blue', 'green', 'red']
    
    for i, (tau, (V, I, L)) in enumerate(lifetime_results.items()):
        color = colors[i % len(colors)]
        label = f'τ = {tau:.0e} s (L = {L:.0f} μm)'
        ax1.semilogy(V, I, color=color, linewidth=2, label=label)
    
    ax1.set_xlabel('Voltage (V)', fontsize=12)
    ax1.set_ylabel('Current (A)', fontsize=12)
    ax1.set_title('Dark I-V: Effect of Carrier Lifetime', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Diffusion length vs lifetime
    tau_values = np.logspace(-7, -4, 50)
    L_values = np.sqrt(D_n * tau_values) * 1e4  # um
    
    ax2.loglog(tau_values * 1e6, L_values, 'b-', linewidth=2)
    ax2.axhline(y=200, color='r', linestyle='--', label='Base thickness = 200 μm')
    ax2.set_xlabel('Lifetime (μs)', fontsize=12)
    ax2.set_ylabel('Diffusion Length (μm)', fontsize=12)
    ax2.set_title('Diffusion Length vs Lifetime', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey insight:")
    print("- Longer lifetime → Longer diffusion length → Better collection")
    print("- For good efficiency: L > Base thickness")
    print("- Lower dark current indicates better material quality")

---

## 5. Surface Recombination Effects

Surface recombination velocity (SRV) significantly impacts solar cell performance, especially for thin cells.

In [ ]:
# Compare surface recombination velocities
srv_values = [1e2, 1e4, 1e6]  # cm/s
srv_results = {}

print("Surface Recombination Comparison:")
print("="*50)

for srv in srv_values:
    print(f"\nSRV = {srv:.0e} cm/s...")
    
    sim = create_solar_cell(
        emitter_depth=0.5,
        base_thickness=200.0,
        emitter_doping=1e19,
        base_doping=1e16,
        device_type='n_on_p',
        temperature=300,
        srh=True,
        taun0=1e-5,
        taup0=1e-5,
        front_surface_velocity=srv,
        back_surface_velocity=srv,
        log_iv=True,
        iv_file=f"dark_srv{int(np.log10(srv))}",
        forward_sweep=(0.0, 0.7, 0.02)
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            V, I = iv.get_iv_data(electrode=2)
            srv_results[srv] = (np.array(V), np.abs(np.array(I)))
            print(f"  Simulation successful")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Simulation failed")

In [ ]:
# Plot SRV comparison
if srv_results:
    plt.figure(figsize=(10, 6))
    
    colors = ['blue', 'green', 'red']
    
    for i, (srv, (V, I)) in enumerate(srv_results.items()):
        color = colors[i % len(colors)]
        label = f'SRV = {srv:.0e} cm/s'
        plt.semilogy(V, I, color=color, linewidth=2, label=label)
    
    plt.xlabel('Voltage (V)', fontsize=12)
    plt.ylabel('Current (A)', fontsize=12)
    plt.title('Dark I-V: Effect of Surface Recombination', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey observations:")
    print("- Lower SRV → Lower dark current → Higher Voc")
    print("- Surface passivation is critical for efficiency")
    print("- SRV < 100 cm/s is excellent passivation")

---

## 6. Doping Optimization

Let's explore how base doping affects solar cell characteristics.

In [ ]:
# Compare different base doping levels
doping_levels = [1e15, 1e16, 1e17]  # cm^-3
doping_results = {}

print("Base Doping Comparison:")
print("="*50)

for Na in doping_levels:
    print(f"\nNa = {Na:.0e} cm⁻³...")
    
    sim = create_solar_cell(
        emitter_depth=0.5,
        base_thickness=200.0,
        emitter_doping=1e19,
        base_doping=Na,
        device_type='n_on_p',
        temperature=300,
        srh=True,
        taun0=1e-5,
        taup0=1e-5,
        log_iv=True,
        log_bands_eq=True,
        iv_file=f"dark_Na{int(np.log10(Na))}",
        forward_sweep=(0.0, 0.7, 0.02)
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            V, I = iv.get_iv_data(electrode=2)
            doping_results[Na] = (np.array(V), np.abs(np.array(I)))
            print(f"  Simulation successful")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Simulation failed")

In [ ]:
# Plot doping comparison
if doping_results:
    plt.figure(figsize=(10, 6))
    
    colors = ['blue', 'green', 'red']
    
    for i, (Na, (V, I)) in enumerate(doping_results.items()):
        color = colors[i % len(colors)]
        label = f'Na = {Na:.0e} cm⁻³'
        plt.semilogy(V, I, color=color, linewidth=2, label=label)
    
    plt.xlabel('Voltage (V)', fontsize=12)
    plt.ylabel('Current (A)', fontsize=12)
    plt.title('Dark I-V: Effect of Base Doping', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nDoping trade-offs:")
    print("- Higher doping → Higher Voc (lower I0)")
    print("- Higher doping → Reduced lifetime (Auger)")
    print("- Optimal doping depends on material quality")

---

## 7. Complete Simulation Example

Let's run a complete solar cell characterization with band diagrams.

In [ ]:
# Complete solar cell characterization
sim_complete = create_solar_cell(
    # Geometry
    emitter_depth=0.5,
    base_thickness=200.0,
    device_width=1.0,
    
    # Mesh
    nx=3,
    ny=150,
    
    # Doping
    emitter_doping=1e19,
    base_doping=1e16,
    device_type='n_on_p',
    
    # Models
    temperature=300,
    srh=True,
    auger=True,
    conmob=True,
    fldmob=True,
    
    # Material
    taun0=1e-5,
    taup0=1e-5,
    
    # Surface recombination (well-passivated)
    front_surface_velocity=1e3,
    back_surface_velocity=1e5,
    
    # Output
    log_iv=True,
    iv_file="solar_complete",
    log_bands_eq=True,
    
    # Dark I-V sweep
    forward_sweep=(0.0, 0.75, 0.01)
)

print("Complete Solar Cell Simulation")
print("="*50)
print("Running simulation...")

result_complete = sim_complete.run()

if result_complete.returncode == 0:
    print("Simulation completed successfully!")
    print(f"\nOutput directory: {sim_complete.working_dir}")
    print("\nOutputs generated:")
    print(sim_complete.outputs.summary())
else:
    print("Simulation failed!")

In [ ]:
# Plot band diagram and I-V
try:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Band diagram
    sim_complete.plot_band_diagram(show=False)
    plt.sca(ax1)
    
    # I-V characteristic
    iv_complete = sim_complete.get_iv_data()
    V_comp, I_comp = iv_complete.get_iv_data(electrode=2)
    V_comp = np.array(V_comp)
    I_comp = np.abs(np.array(I_comp))
    
    ax2.semilogy(V_comp, I_comp, 'b-', linewidth=2)
    ax2.set_xlabel('Voltage (V)', fontsize=12)
    ax2.set_ylabel('Current (A)', fontsize=12)
    ax2.set_title('Complete Dark I-V Characteristic', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Could not create plots: {e}")
    # Try individual plots
    sim_complete.plot_band_diagram(title="Solar Cell Band Diagram")

---

## 8. Exercises

### Exercise 1: P-on-N Solar Cell
Create a P-on-N structure and compare it to the N-on-P design.

In [ ]:
# Exercise 1: P-on-N solar cell
sim_pon = create_solar_cell(
    emitter_depth=0.5,
    base_thickness=200.0,
    emitter_doping=1e19,
    base_doping=1e16,
    device_type='p_on_n',  # P-on-N structure
    temperature=300,
    srh=True,
    taun0=1e-5,
    taup0=1e-5,
    log_iv=True,
    forward_sweep=(0.0, 0.7, 0.02)
)

print("Running P-on-N simulation...")
result_pon = sim_pon.run()

if result_pon.returncode == 0:
    print("Simulation completed!")
    
    try:
        iv_pon = sim_pon.get_iv_data()
        V_pon, I_pon = iv_pon.get_iv_data(electrode=2)
        
        plt.figure(figsize=(10, 6))
        plt.semilogy(V_pon, np.abs(I_pon), 'r-o', linewidth=2, label='P-on-N')
        
        # Compare with N-on-P if available
        if 'V_dark' in dir() and 'I_dark' in dir():
            plt.semilogy(V_dark, I_dark, 'b-', linewidth=2, label='N-on-P')
        
        plt.xlabel('Voltage (V)', fontsize=12)
        plt.ylabel('Current (A)', fontsize=12)
        plt.title('N-on-P vs P-on-N Solar Cell', fontsize=14)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("\nN-on-P vs P-on-N:")
        print("- P-on-N has P+ emitter, N base")
        print("- Better for high-energy radiation (space applications)")
        print("- Minority carrier collection differs")
        
    except Exception as e:
        print(f"Could not plot: {e}")
else:
    print("Simulation failed")

---

## Summary

In this notebook, you learned:

1. **Solar Cell Structure**: N-on-P junction with emitter and base
2. **Running PADRE Simulations**: Using `create_solar_cell()` and `sim.run()`
3. **Dark I-V Analysis**: Understanding recombination from simulation
4. **Lifetime Effects**: Diffusion length and collection efficiency
5. **Surface Recombination**: Importance of passivation
6. **Doping Optimization**: Trade-offs in base doping

**Key Equations:**
- $V_{oc} = \frac{kT}{q}\ln(I_L/I_0 + 1)$
- $L = \sqrt{D\tau}$ (diffusion length)
- For good collection: L > Base thickness

**Next**: [07 - MOS Capacitor](07_MOS_Capacitor.ipynb) - Capacitance-voltage analysis